In [1]:
pip install earthengine-api


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 23.7 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install geemap

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 30.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 34.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.9 MB/s 

In [ ]:
import ee
import geemap
import datetime

In [2]:


# Initialize the Earth Engine module
ee.Authenticate()

ee.Initialize()


In [3]:
import ee
import geemap
import folium

# Define the region of interest - Tashkent, Uzbekistan
tashkent = ee.Geometry.Rectangle([69.0, 41.15, 69.4, 41.4])

# Function to mask clouds using the QA60 band
def maskS2clouds(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    # Both flags should be set to zero, indicating clear conditions
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
        qa.bitwiseAnd(cirrusBitMask).eq(0)
    )
    return image.updateMask(mask).divide(10000)

# Select bands for classification
bands = ['B2', 'B3', 'B4', 'B8']  # Blue, Green, Red, NIR

# Define training samples with labeled classes

# Water class (0)
water = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([69.25, 41.28]), {'landcover': 0}),
    ee.Feature(ee.Geometry.Point([69.20, 41.22]), {'landcover': 0}),
    # Add more water points
])

# Urban class (1)
urban = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([69.30, 41.32]), {'landcover': 1}),
    ee.Feature(ee.Geometry.Point([69.33, 41.35]), {'landcover': 1}),
    # Add more urban points
])

# Vegetation class (2)
vegetation = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([69.10, 41.25]), {'landcover': 2}),
    ee.Feature(ee.Geometry.Point([69.15, 41.27]), {'landcover': 2}),
    # Add more vegetation points
])

# Agriculture class (3)
agriculture = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([69.05, 41.20]), {'landcover': 3}),
    ee.Feature(ee.Geometry.Point([69.08, 41.22]), {'landcover': 3}),
    # Add more agriculture points
])

# Merge all training samples into one FeatureCollection
trainingSamples = water.merge(urban).merge(vegetation).merge(agriculture)

# Function to create classified image for a given year
def classifyYear(year):
    # Define date range for the year
    startDate = ee.Date.fromYMD(year, 1, 1)
    endDate = ee.Date.fromYMD(year, 12, 31)

    # Load Sentinel-2 surface reflectance image collection for the year
    sentinel2 = (
        ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(tashkent)
        .filterDate(startDate, endDate)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        .map(maskS2clouds)
    )

    # Create a median composite image
    composite = sentinel2.median().clip(tashkent)

    # Sample the composite image at the locations of the training samples
    training = composite.select(bands).sampleRegions(
        collection=trainingSamples,
        properties=['landcover'],
        scale=10
    )

    # Train a Random Forest classifier with default parameters
    classifier = ee.Classifier.smileRandomForest(50).train(
        features=training,
        classProperty='landcover',
        inputProperties=bands
    )

    # Classify the composite image
    classified = composite.select(bands).classify(classifier)

    # Return the classified image
    return classified

# Define visualization parameters
palette = ['0000FF', 'FF0000', '00FF00', 'FFFF00']  # Blue, Red, Green, Yellow
vizParams = {
    'min': 0,
    'max': 3,
    'palette': palette,
    'format': 'png'
}

# Create classified images for each year
classified2020 = classifyYear(2020)
classified2021 = classifyYear(2021)
classified2022 = classifyYear(2022)

# Create a map centered on Tashkent
Map = geemap.Map(center=[41.3, 69.25], zoom=10)

# Add layers to the map
Map.addLayer(classified2020, vizParams, 'Land Use Classification 2020')
Map.addLayer(classified2021, vizParams, 'Land Use Classification 2021')
Map.addLayer(classified2022, vizParams, 'Land Use Classification 2022')

# Display the map
Map


/Users/nodiratillayeva/Desktop/Change detection/.venv/lib/python3.9/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)


Map(center=[41.3, 69.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

Matplotlib is building the font cache; this may take a moment.


In [4]:
Map.save('tashkent_ndvi_change_2018_2024.html')
